In [37]:
# 读取数据
import pandas as pd
df = pd.read_csv('../data/SeattleWeather_unclean.csv')
df

,STATION,NAME,DATE,AWND,PRCP,TMAX,TMIN,WT01,WT02,WT03,WT08
0,USW00024234,"SEATTLE BOEING FIELD, WA US",2015-01-01,0.22,0.00,44.0,25.0,NaN,NaN,NaN,1.0
1,USW00024234,"SEATTLE BOEING FIELD, WA US",2015-01-02,4.47,0.03,44.0,30.0,1.0,NaN,NaN,NaN
2,USW00024234,"SEATTLE BOEING FIELD, WA US",2015-01-03,2.91,0.01,44.0,34.0,1.0,NaN,NaN,1.0
3,USW00024234,"SEATTLE BOEING FIELD, WA US",2015-01-04,8.28,0.21,54.0,40.0,1.0,NaN,NaN,1.0
4,USW00024234,"SEATTLE BOEING FIELD, WA US",2015-01-05,11.41,0.07,58.0,52.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2759,USW00024234,"SEATTLE BOEING FIELD, WA US",2022-07-30,3.80,0.00,92.0,66.0,NaN,NaN,NaN,NaN
2760,USW00024234,"SEATTLE BOEING FIELD, WA US",2022-07-31,4.03,0.00,93.0,65.0,NaN,NaN,NaN,NaN
2761,USW00024234,"SEATTLE BOEING FIELD, WA US",2022-08-01,5.82,0.00,81.0,62.0,NaN,NaN,NaN,NaN
2762,USW00024234,"SEATTLE BOEING FIELD, WA US",2022-08-02,6.04,0.00,77.0,59.0,NaN,NaN,NaN,NaN


In [38]:
# Clean the raw data from NOAA
# WT01 雾
# WT02 大雾
# WT03 打雷
# WT08 霾

# 华氏度转摄氏度；综合天气分析

import time
import datetime

df1=df.drop('STATION', axis=1)
df1=df1.drop('NAME', axis=1)
df1=df1.dropna(axis=0, how='any', subset=['AWND', 'PRCP', 'TMAX', 'TMIN']) #删除缺失必要数据的行

datestamparray = []
datearray = []
precarray = []
tmaxarray = []
tminarray = []
windarray = []
weatherarray = []

for row in df1.iterrows():
    dateinfo = row[1]['DATE'] + ' 16:00:00'
    datearray.append(dateinfo)
    # dateinfo = dateinfo.replace('-','/')
    datestamparray.append(time.mktime(datetime.datetime.strptime(dateinfo,"%Y-%m-%d %H:%M:%S").timetuple()))
    windarray.append(row[1]['AWND'])
    precarray.append("{:.2f}".format(row[1]['PRCP'] * 25.4))
    tmaxarray.append("{:.2f}".format((row[1]['TMAX'] - 32) * 5.0/9.0))
    tminarray.append("{:.2f}".format((row[1]['TMIN'] - 32) * 5.0/9.0))

    if row[1]['PRCP'] <= 0.2 and row[1]['PRCP'] > 0.03 and row[1]['TMIN'] > 32:
        weatherarray.append('drizzle')
    elif row[1]['PRCP'] > 0.2 and row[1]['TMIN'] > 32:
        weatherarray.append('rain')
    elif row[1]['PRCP'] > 0.03 and row[1]['TMIN'] <= 26 and row[1]['TMAX'] <= 50:
        weatherarray.append('snow')
    elif row[1]['WT01'] == 1 or row[1]['WT02'] == 1 or row[1]['WT08'] == 1:
        weatherarray.append('fog')
    else:
        weatherarray.append('sun')

data = {
    'id': datestamparray,
    'date': datearray,
    'precipitation': precarray,
    'temp_max':tmaxarray,
    'temp_min':tminarray,
    'wind': windarray,
    'weather':weatherarray
}

datadf = pd.DataFrame(data)
# print(datadf)

path = '../data/SeattleWeather_cleaned.csv'
datadf.to_csv(path,sep=',',index=False,header=True)


TypeError: Tuple or struct_time argument required